In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://mongodb:27017/')
db = client.metro_lisboa
collection = db.estacoes

# Print the first document
print(collection.find_one())

# Close the connection
client.close()

In [ ]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

spark = SparkSession.builder \
    .appName("Metro de Lisboa") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/metro_lisboa.estacoes") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/metro_lisboa.estacoes") \
    .getOrCreate()

df = spark.read.format("mongo").load()

stops_per_line_df = df.select("properties.LINHA")

stops_per_line = stops_per_line_df.groupBy("LINHA").count()

stops_per_line_pd = stops_per_line.toPandas()

plt.figure(figsize=(10,6))
plt.bar(stops_per_line_pd['LINHA'], stops_per_line_pd['count'], color='blue')
plt.xlabel('Linha de Metro')
plt.ylabel('Número de Paragens')
plt.title('Número de Paragens por linha de Metro')
plt.xticks(rotation=45)  
plt.tight_layout()       
plt.show()

spark.stop()